### 03 Estimator 이해 및 fit(), predict() 메서드

 사이킷런은 API 일관성과 개발 편의성을 제공하기 위한 노력이 엿보이는 패키지이다. 사이킷런은 ML모델 학습을 위해서 fit()을, 학습된 모델의 예측을 위해 predict()메서드를 제공한다. 지도학습의 주요 두 축인 분류(Classification)와 회귀(Regression)의 다양한 알고리즘을 구현한 모든 사이킷런 클래스는 fit()과 predict()만을 이용해 간단하게 학습과 예측 결과를 반환한다. 

사이팃런에서는 분류 알고리즘을 구현한 클래스를 Classifier로, 그리고 회귀 알고리즘을 구현한 클래스를 Regressor 로 지칭한다. 이들 Classifier와 Regressor를 합쳐서 Estimator 클래스라고 부른다. 즉, 지도학습의 모든 알고리즘을 구현한 클래스를 통칭해서 Estimator라고 부른다.

당연히 Estimator 클래스는 fit()과 predict()를 내부에서 구현하고 있다.

사이킷런에서 비지도학습인 차원 축소, 클러스터링, 피처 추출(Feature Extraction) 등을 구현한 클래스 역시 대부분 fit()과 transform()을 적용한다. 비지도학습과 피처 추출에서 fit()은 지도학습의 fit()과 같이 학습을 의미하는 것이 아니라 입력 데이터의 형태에 맞춰 데이터를 변환하기 위한 사전 구조를 맞추는 작업이다.  

In [2]:
from sklearn.datasets import load_iris

iris_data = load_iris()
print(type(iris_data))

<class 'sklearn.utils.Bunch'>


load_iris() API의 반환결과가 sklearn.utils.Bunch 클래스임을 의미함. Bunch클래스는 기초 파이썬의 딕셔너리 형태와 매우 유사하다 

In [4]:
keys = iris_data.keys()
print('붓꽃 데이터 세트의 키들:', keys)

붓꽃 데이터 세트의 키들: dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])


In [9]:
print('\n feature_names의 type:', type(iris_data.feature_names))
print('\n feature_names의 shape:', len(iris_data.feature_names))
print(iris_data.feature_names)

print('\n target_names의 type:', type(iris_data.target_names))
print('\n tatget_names의 shape:', len(iris_data.target_names))
print(iris_data.target_names)

print('\n target의 type:', type(iris_data.target))
print('\n tatget의 shape:', len(iris_data.target))
print(iris_data.target)


 feature_names의 type: <class 'list'>

 feature_names의 shape: 4
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']

 target_names의 type: <class 'numpy.ndarray'>

 tatget_names의 shape: 3
['setosa' 'versicolor' 'virginica']

 target의 type: <class 'numpy.ndarray'>

 tatget의 shape: 150
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [10]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np

In [11]:
dt_clf = DecisionTreeClassifier()
iris_data = load_iris()

In [13]:
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target,\
                                                   test_size=0.3, random_state=121)

In [15]:
X_test.shape

(45, 4)

In [16]:
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print('예측 정확도:{0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도:0.9556


#### 교차 검증

앞서 알고리즘을 학습시키는 학습데이터와 이에 대한 예측 성능을 평가하기 위한 별도의 테스트용 데이터가 필요하다고 하였다(hold_out). 하지만 이 방법 역시 과적합(overfitting)에 취약한 약점을 가질 수 있다.


* 과적합이란 모델이 학습 데이터에만 과도하게 최적화되어, 실제 예측을 다른 데이터로 수행할 경우에는 예측 성능이 과도하게 떨어지는 것을 말한다. 


그런데 고정된 학습 데이터와 테스트 데이터로 평가를 하다보면 테스트 데이터에만 최적의 성능을 발휘할 수 있도록 편향되게 모델을 유도하는 경향이 생기게 된다. 이러한 문제점을 개선하기 위해 교차 검증을 이용해 더 다양한 학습과 평가를 수행한다.

In [17]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd

In [18]:
iris = load_iris()
features = iris.data #x값
label = iris.target #y값
dt_clf = DecisionTreeClassifier(random_state=156)

#5개의 폴드 세트로 분리하는 KFold 객체와 폴드 세트별 정확도를 담을 리스트 객체 생성.
Kfold = KFold(n_splits=5)
print('붓꽃 데이터 세트 크기:', features.shape[0])

붓꽃 데이터 세트 크기: 150
